In [ ]:
%load_ext watermark


In [ ]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp
from tqdm import tqdm

from pylib._seed_global_rngs import seed_global_rngs


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "2025-04-30-profile-mut-screen"
teeplot_subdir


In [ ]:
seed_global_rngs(1)


## Get Data


In [ ]:
df = pd.read_parquet("https://osf.io/ag5w7/download")


In [ ]:
df["replicate_uuid"].nunique()


In [ ]:
df["is_focal_mutation"] = df["mut_char_pos"] == 0


In [ ]:
stats = (
    "binom_p",
    "binom_stat",
    "trinom_p",
    "trinom_stat",
    "trinom_p_fill0",
    "trinom_stat_fill0",
    "mw_p",
    "cliffs_delta",
    "mw_p_dropna",
    "cliffs_delta_dropna",
)


In [ ]:
records = []
groups = df[
    (df["trt_hsurf_bits"] == 0)
    & (df["trt_n_downsample"] == df["trt_n_downsample"].max())
    & (df["tb_stat"].str.contains("ratio"))
].groupby(
    ["trt_name", "replicate_uuid", "screen_uuid", "screen_name", "tb_stat"],
    observed=True,
)
for (trt_name, replicate_uuid, screen_uuid, screen_name, tb_stat), grp in tqdm(
    groups,
):
    grp_focal = grp[grp["is_focal_mutation"]]
    grp_nonfocal = grp[~grp["is_focal_mutation"]]

    record = {
        "trt_name": trt_name,
        "replicate_uuid": replicate_uuid,
        "screen_uuid": screen_uuid,
        "screen_name": screen_name,
        "tb_stat": tb_stat,
    }

    records.append(
        {
            **record,
            **{
                stat: (
                    np.nan if len(grp_focal) == 0 else grp_focal[stat].item()
                )
                for stat in stats
            },
            **{"mutation": "focal"},
        },
    )
    for __ in range(1_000):
        records.append(
            {
                **record,
                **{
                    stat: grp_nonfocal[stat].sample(1).item() for stat in stats
                },
                **{"mutation": "nonfocal"},
            },
        )


In [ ]:
for stat in stats:
    display(HTML(f"<h3>{stat}</h3>"))
    with tp.teed(
        sns.catplot,
        data=pd.DataFrame(records),
        x="trt_name",
        y=stat,
        hue="mutation",
        row="screen_name",
        col="tb_stat",
        kind="box",
        height=2,
        aspect=1.5,
        margin_titles=True,
    ):
        pass


In [ ]:
df.loc[
    (df["trt_name"] == "Sben/Gneu")
    & (df["mut_char_pos"] == 0)
    & (df["tb_stat"].str.endswith("ratio")),
    [
        "mut_nobs",
        "mut_freq",
        "binom_n",
        "binom_k",
        "binom_p",
        "mw_p_dropna",
        "tb_stat",
        "screened_N",
        "trt_name",
        "mut_char_pos",
        "screened_nanmean",
    ],
].sample(25)
